In [23]:
from pathlib import Path
import pandas as pd
import geopandas as gpd
from mtcpy.aws import pull_df_from_redshift_sql, pull_geotable_redshift

In [24]:
# set workign directory 
work_dir = Path("/Users/jcroff/Library/CloudStorage/Box-Box/DSA Projects/Spatial Analysis and Mapping/Wildfire Risk Analysis/Data Outputs")

# pull data from redshift

In [ ]:
req_cols = [
    "geoid",
    # "zoning_type", - out of scope
    # "zoning_subtype", - out of scope
    "structno",
    "yearbuilt",
    "numstories",
    "numunits",
    # "numrooms", - out of scope
    # "structstyle", - available in premium but not ingested into redshift
    # "improvval", - out of scope
    # "landval", - out of scope
    # "parval", - out of scope
    "saleprice",
    "scity",
    "county",
    "szip",
    "lat",
    "lon",
    "fema_flood_zone",
    # "fema_nri_risk_rating", - available in premium but not ingested into redshift
    # "qoz", - avialable in premium but not ingested into redshift
    # "census_tract", - out of scope
    # "census_block", - out of scope
    # "sourceurl", - available in premium but not ingested into redshift
    # "recrdareano", - available in premium but not ingested into redshift
    "ll_gisacre",
    "ll_gissqft",
    "ll_bldg_footprint_sqft",
    "ll_uuid",
    "ll_stack_uuid",
    "ll_bldg_count",
    "lbcs_structure_desc",
    # "housing_affordability_index", - available in premium but not ingested into redshift
    # "population_density", - available in premium but not ingested into redshift
    # "population_growth_past_5_years", - available in premium but not ingested into redshift
    # "population_growth_next_5_years", - available in premium but not ingested into redshift
    # "housing_growth_past_5_years", - available in premium but not ingested into redshift
    # "housing_growth_next_5_years", - available in premium but not ingested into redshift
    # "median_household_income", - available in premium but not ingested into redshift
    # "transmission_line_distance", - available in premium but not ingested into redshift
]
sql_cols = ", ".join(req_cols)
sql = f"SELECT {sql_cols} FROM regrid_v25.parcel_source_tbl_v25" 
df = pull_df_from_redshift_sql(sql_statement=sql)

took 1.4165 minutes


In [42]:
# convert numeric columns to numeric
numeric_cols = [
    "structno",
    "yearbuilt",
    "numstories",
    "numunits",
]
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col])

In [43]:
df["numunits"].isna().mean()

# overall, .80 of the parcels is missing numunits

0.8002539159719686

In [37]:
# get a count of records where structno is equal to or greater than 1
df[df["structno"] > 0].shape[0]

159013

In [44]:
# get a count of records where ll_bldg_count is equal to or greater than 1
df[df["ll_bldg_count"] > 0].shape[0]

2168498

In [38]:
# get a count of records where numunits is not null but structno is null
df[(df["numunits"].notnull()) & (df["structno"].isnull())].shape[0]

311022

In [45]:
# get a count of records where numunits is not null but ll_bldg_count is null
df[(df["numunits"].notnull()) & (df["ll_bldg_count"].isnull())].shape[0]

0

In [ ]:
# get a summary of non-null numunits by lbcs_structure_desc
grouped = df.groupby("lbcs_structure_desc")["numunits"].agg(
    count_non_null_units="count",  # non-null values
    total_number_structures="size",  # total number of rows in group
    missing_units=lambda x: x.isna().sum(),  # number of missing
    missing_pct_units=lambda x: x.isna().mean() * 100,  # percent missing
)
grouped

,count_non_null_units,total_number_structures,missing_units,missing_pct_units
lbcs_structure_desc,,,,
Air and space transportation facility,0,20,20,100.000000
Airport terminal,0,19,19,100.000000
Assembly and construction-type plants,0,216,216,100.000000
Attached units,10858,10876,18,0.165502
Automobile parking facilities,4,1532,1528,99.738903
...,...,...,...,...
Utility and other nonbuilding structures,0,1075,1075,100.000000
Warehouse discount store building,1,10,9,90.000000
Warehouse or storage facility,106,6694,6588,98.416492


In [27]:
grouped.sort_values(by='missing_pct_units', ascending=True).reset_index().to_csv(
    work_dir / "missing_numunits_by_structure_type.csv",
    index=False
)

In [17]:
df.lbcs_structure_desc.unique()

array(['Residential buildings', 'Unclassified', 'Single-family buildings',
       'Industrial buildings and structures',
       'Public assembly structures', 'Multifamily structures: Four Units',
       'Water-supply-related facility', 'Townhouses',
       'Multifamily structures', 'Duplex structures',
       'Store or shop building', 'Restaurant building',
       'Hotels, motels, and tourist courts', 'Department store building',
       'Warehouse or storage facility',
       'Light industrial structures and facilities', 'Highways and roads',
       'Office or store building with residence on top',
       'Commercial buildings and other specialized structures',
       'Office or bank building', 'School or university buildings',
       'Automobile repair and service structures', 'Manufactured housing',
       'Churches, synagogues, temples, mosques, etc.', 'Greenhouses',
       'Multifamily structures: Three Units',
       'Multifamily structures: Two Units', 'Medical facility',
       

In [48]:
# look at single-family building missing numunits
df.query("lbcs_structure_desc == 'Single-family buildings'")

,geoid,structno,yearbuilt,numstories,numunits,saleprice,scity,county,szip,lat,lon,fema_flood_zone,ll_gisacre,ll_gissqft,ll_bldg_footprint_sqft,ll_uuid,ll_stack_uuid,ll_bldg_count,lbcs_structure_desc
1,06001,NaN,NaN,NaN,NaN,NaN,Piedmont,Alameda,94611-3606,37.815214,-122.218492,X,0.23,10043.0,2712.0,e67b4a4b-068f-456f-81dc-68e0c3da6d95,None,1,Single-family buildings
2,06001,NaN,NaN,NaN,NaN,NaN,Piedmont,Alameda,94611-3606,37.815012,-122.218161,X,0.39,16915.0,4972.0,53e373c8-27bb-4e59-b93b-f23bbaa89c59,None,1,Single-family buildings
3,06001,NaN,NaN,NaN,NaN,NaN,Piedmont,Alameda,94610-1211,37.814599,-122.224538,X,0.22,9445.0,2231.0,a8be29bf-090b-4401-be9a-56570f6af07a,None,2,Single-family buildings
4,06001,NaN,NaN,NaN,NaN,NaN,Piedmont,Alameda,94611-3437,37.825140,-122.220374,X,0.07,3102.0,NaN,60ec0c13-b401-4b17-9b03-bf7523ec11fd,None,0,Single-family buildings
7,06001,NaN,NaN,NaN,NaN,NaN,Piedmont,Alameda,94611-3314,37.816089,-122.213967,X,0.48,20849.0,3465.0,5d7b4d4f-97bd-4e1c-bf37-3639417fae17,None,1,Single-family buildings
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2324330,06097,1.0,1927.0,1.0,1.0,185000.0,Santa Rosa,Sonoma,95407-7757,38.386830,-122.733277,X,0.49,21187.0,4819.0,c5222228-2a2f-42dd-b2ed-dc52225b98c6,None,5,Single-family buildings
2324382,06097,1.0,1964.0,1.0,1.0,650000.0,Santa Rosa,Sonoma,95409-3512,38.469401,-122.662494,X,0.16,7153.0,2294.0,f1cf800b-a81d-45fe-817c-076b9c1bd0b3,None,1,Single-family buildings
2324383,06097,1.0,1964.0,1.0,1.0,585000.0,Santa Rosa,Sonoma,95409-3561,38.469394,-122.662788,X,0.16,7168.0,2746.0,bb826a59-c0ef-4dc7-ad05-6178dbf5f01d,None,1,Single-family buildings
2324384,06097,1.0,1963.0,1.0,1.0,600000.0,Santa Rosa,Sonoma,95409-3561,38.469345,-122.663000,X,0.17,7317.0,2913.0,9fdfaa91-f612-4bae-b795-3cfa54887ee1,None,1,Single-family buildings
